In [1]:
import pandas as pd
import MySQLdb
import datetime
import os;

In [2]:
db = MySQLdb.connect("localhost","root","123456","i_day_data")
db.set_character_set("gbk")

In [104]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [3]:
B1_list = [['K','U'],['U','F'],['F','K']] #鐵礦石 2013-10-18 / 2017-11-18 
# B1_list = [['F','K'],['K','U'],['U','F']] #玉米2004-10-29
# B1_list = [['U','F'],['F','K'],['K','U']] #焦炭 2011-04-15 / 焦煤 2013-03-22 / 豆粕2000-7-17 
# B1_list = [['M','Z'],['Z','M']] #黃金 2008-01-09 NO B2
# B1_list = [['Z','M'],['M','Z']] #白銀 2012-05-10 NO B2

In [4]:
B2_list = ['U','F','K'] #鐵礦石
# B2_list = ['K','U','F'] #玉米
# B2_list = ['F','K','U'] #焦炭/焦煤/豆粕

In [8]:
start_date = '2013-10-18'
contract_id = "IOE"

change_date = conti_contract_B1(start_date,contract_id,B1_list,"i_B1.csv",14,19)
conti_contract_B2(start_date,contract_id,change_date,B2_list,"i_B2.csv",14,19)

2014-02-26
tbname_1: IOEK14 start_date: 2013-10-18 end_date: 2014-02-26
2014-07-22
tbname_1: IOEU14 start_date: 2014-02-27 end_date: 2014-07-22
2014-10-22
tbname_1: IOEF15 start_date: 2014-07-23 end_date: 2014-10-22
2015-03-06
tbname_1: IOEK15 start_date: 2014-10-23 end_date: 2015-03-06
2015-07-20
tbname_1: IOEU15 start_date: 2015-03-07 end_date: 2015-07-20
2015-11-20
tbname_1: IOEF16 start_date: 2015-07-21 end_date: 2015-11-20
2016-03-18
tbname_1: IOEK16 start_date: 2015-11-21 end_date: 2016-03-18
2016-08-10
tbname_1: IOEU16 start_date: 2016-03-19 end_date: 2016-08-10
2016-11-24
tbname_1: IOEF17 start_date: 2016-08-11 end_date: 2016-11-24
2017-03-21
tbname_1: IOEK17 start_date: 2016-11-25 end_date: 2017-03-21
2017-07-31
tbname_1: IOEU17 start_date: 2017-03-22 end_date: 2017-07-31
2017-11-22
tbname_1: IOEF8 start_date: 2017-08-01 end_date: 2017-11-22
1
tbname_1: IOEK8 start_date: 2017-11-23 end_date: 2018-02-01


# 主力合約 (B)

In [5]:
def conti_contract_B1(start_date,contract_id,B_list,filename,start_year,end_year):
    contract = pd.DataFrame()
    contract_name = []
    change = [1]
    change_date = []

    for year in range(start_year,end_year):
        #修改合約順序
        
        if(year<10):
            str_year = "0" + str(year)
        elif(year==18):
            str_year = "8"
        else:
            str_year = str(year)
            
        for month in B_list:
            cursor = db.cursor()
     
            tbname_1 = contract_id + month[0] + str_year
            
            if(month[0]=="Z"):
                year+=1             
                str_year = str(year)
                
                #因為目前資料只到17年
                if(year==19):
                    break;
                    
                if(len(str_year)<2):
                    str_year = "0" + str_year 
            
            if(month[1]=="F"):
                year+=1
                str_year = str(year)
                
                if(year==19):
                    break;
                
                if(len(str_year)<2):
                    str_year = "0" + str_year  
                    
            if(str_year=="18"):
                str_year = "8"

            tbname_2 = contract_id + month[1] + str_year

                                
            #print(tbname_2)
            #找換約日期
            sql = "SELECT table1.Date FROM " + tbname_1 + " table1," + tbname_2 + " table2" +\
                  " WHERE table2.OI > table1.OI AND table1.Date = table2.Date AND table2.Date > '" + start_date + "' limit 1"
            cursor.execute(sql)
            date = cursor.fetchone()

            if(start_date >= '2018-01-09' ):
                continue
                
            elif(date == None):
                print(1)
                sql = "SELECT Date FROM " + tbname_1 +" order by Date desc limit 1"
                cursor.execute(sql)
                date = cursor.fetchone()
                end_date = date[0].strftime('%Y-%m-%d')
            
            else:
                end_date = date[0].strftime('%Y-%m-%d')
                print(end_date)
            
            print("tbname_1: " + tbname_1,"start_date: " + start_date,"end_date: " + end_date)
            change_date.append(end_date)
            
            #資料讀取
            sql = "SELECT * FROM " + tbname_1 + " WHERE Date <= '" + end_date + "' and Date>='" + start_date + "'"

            cursor.execute(sql)
            db.commit()
            data = cursor.fetchall()

            for i in range(len(list(data))):
                contract_name.append(tbname_1)
                change.append(0)

            contract = contract.append(pd.DataFrame(list(data)))
            change.pop()
            change.append(1)

            start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
            
            
    change.pop()
    contract['contract_name'] = contract_name
    contract['change'] = change
    
    #存檔
    columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
    contract.to_csv('../日資料/conti/B/' + filename,index=None,header=columns)
    
    return change_date

# 其他合約 (B)

In [6]:
def conti_contract_B2(start_date,contract_id,change_date,B_list,filename,start_year,end_year):
    contract = pd.DataFrame()
    contract_name = []
    change = [1]
    date_count = 0

    for year in range(start_year,end_year):
            #修改合約順序
            
        if(year<10):
            str_year = "0" + str(year)
        else:
            str_year = str(year)
            
        for month in B_list:
            cursor = db.cursor()

            if(month=="F"):
                year+=1
                str_year = str(year)
                
                if(year==19):
                    break;
                    
                if(len(str_year)<2):
                    str_year = "0" + str_year  
                    
            if(str_year=="18"):
                str_year = "8"
                
            tbname = contract_id + month[0] + str_year

            if(date_count < len(change_date)):
                end_date = change_date[date_count]
            else:
                end_date = "2018-01-09"

                #資料讀取
            sql = "SELECT * FROM " + tbname + " WHERE Date <= '" + end_date + "' and Date>='" + start_date + "'"

            cursor.execute(sql)
            db.commit()
            table1_data = cursor.fetchall()


            for i in range(len(list(table1_data))):
                contract_name.append(tbname)
                change.append(0)

            contract = contract.append(pd.DataFrame(list(table1_data)))
            change.pop()
            change.append(1)
            

            #讀取下一個合約的資料
            date_count += 1

            start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

    change.pop()
    contract['contract_name'] = contract_name
    contract['change'] = change
    
    

    #存檔
    columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
    contract.to_csv('../日資料/conti/B/' + filename,index=None,header=columns)

# 螺紋鋼

In [2]:
db = MySQLdb.connect("localhost","root","123456","rb_day_data")
db.set_character_set("gbk")

In [57]:
# B1_list = [['U','V'],['V','X'],['X','Z'],['Z','F']] #螺紋鋼 2009-03-27 (2009 9 10 11 12 1)
# B1_list = [['F','G'],['G','H'],['H','K']] #螺紋鋼 2009-10-10 (2010 1 2 3 5)
B1_list = [['K','V'],['V','F'],['F','K']] #螺紋鋼 2009-12-23 2010以後

In [30]:
# A1_list = ['U','V','X','Z','F']   #9-10
# A1_list = ['F','G','H','K']     #10-11
# A1_list = ['K','V','F','K']     #11-19

# A2_list = ['V','X','Z','F']
# A2_list = ['G','H','K']
# A2_list = ['V','F','K']

In [33]:
change_date

[]

In [31]:
# contract = pd.DataFrame()
# contract_name = []
# change = [1]
# change_date = []
# date_count = 0

for year in range(11,11):
        
        if(year<10):
            str_year = "0" + str(year)
        elif(year==18):
            str_year = "8"
        else:
            str_year = str(year)
            
        for month in A2_list:
            cursor = db.cursor()

        #有包含Z的話以下要註解掉   
            if(month=="F"):
                year+=1

                if(year < 10):
                    str_year = "0" + str(year) 
                else:
                    str_year = str(year)
                    
                if(str_year=="18"):
                    str_year = "8"
            
            #之後須砍掉
            if(year > 18):
                break
                
            tbname = 'rbt' + month + str_year
            
            if(month=="Z"):
                year += 1
                
                if(year < 10):
                    str_year = "0" + str(year)
                else:
                    str_year = str(year)
                
                if(str_year=="18"):
                    str_year = "8"

            if(date_count < len(change_date)):
                end_date = change_date[date_count]
                date_count += 1
            else:
                end_date = "2018-01-09"      

            #獲取資料
            sql = "SELECT * FROM " + tbname + " WHERE Date >= '" + start_date + "' and Date <= '" + end_date + "'"
            cursor.execute(sql)
            data = cursor.fetchall()
            start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

            
            
            for i in range(len(data)):
                contract_name.append(tbname)
                change.append(0)
            
            contract = contract.append(pd.DataFrame(list(data)))
            change.pop()
            change.append(1)

# change.pop()
# contract['contract_name'] = contract_name
# contract['change'] = change

# #存檔
# columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
# contract.to_csv('../日資料/conti/A/' + filename ,index=None,header=columns)

ValueError: Writing 2 cols but got 10 aliases

In [ ]:
start_date = '2009-12-23'
contract_id = "RBT"

# contract = pd.DataFrame()
# contract_name = []
# change = [1]
# change_date = []

for year in range(10,18):
    #合約順序
    for month in B1_list:
        cursor = db.cursor()
        
        if(year==9):
            tbname_1 = contract_id + month[0] + "09"
        else:
            tbname_1 = contract_id + month[0] + str(year)

        if(month[1] == 'F'):
#             year = 10   #2009
            year += 1
            if(year==18):
                year = 8
                
        if(year==9):
            tbname_2 = contract_id + month[1] + "09"
        else:
            tbname_2 = contract_id + month[1] + str(year)
            
        
        #找換約日期
        sql = "SELECT table1.Date FROM " + tbname_1 + " table1," + tbname_2 + " table2" +\
                " WHERE table2.OI > table1.OI AND table1.Date = table2.Date AND table2.Date >= '" + start_date + "' limit 1"
        cursor.execute(sql)
        date = cursor.fetchone()        

        if(start_date >= '2017-11-17' ):
                continue
                
        elif(date == None):
            print(1)
            sql = "SELECT Date FROM " + tbname_1 +" order by Date desc limit 1"
            cursor.execute(sql)
            date = cursor.fetchone()
            end_date = date[0].strftime('%Y-%m-%d')
            
        else:
            end_date = date[0].strftime('%Y-%m-%d')
            
        change_date.append(end_date)
        
        print("tbname_1: " + tbname_1,"start_date: " + start_date,"end_date: " + end_date)

        #資料讀取
        sql = "SELECT * FROM " + tbname_1 + " WHERE Date <= '" + end_date + "' and Date>='" + start_date + "'"

        cursor.execute(sql)
        data = cursor.fetchall()

        for i in range(len(list(data))):
            contract_name.append(tbname_1)
            change.append(0)

        contract = contract.append(pd.DataFrame(list(data)))
        change.pop()
        change.append(1)

        start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
                   
        
        
change.pop()
contract['contract_name'] = contract_name
contract['change'] = change

#存檔
columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
contract.to_csv('../日資料/conti/rb_B1.csv',index=None,header=columns)

In [62]:
# B2_list = ['V','X','Z','F'] #螺紋鋼 2009-03-27
# B2_list = ['G','H','K'] #螺紋鋼 2009-10-10 (2010 1 2 3 5)
B2_list = ['V','F','K'] #螺紋鋼 2009-12-23 2010以後

In [63]:
start_date = '2009-12-23'
contract_id = "RBT"
len_count = 31

# contract_B2 = pd.DataFrame()
# contract_name_B2 = []
# change_B2 = [1]
date_count = 7

for year in range(10,18):
            #修改合約順序
            
    if(year<10):
            str_year = "0" + str(year)
    else:
        str_year = str(year)
            
    for month in B2_list:
        cursor = db.cursor()

        if(month=="F"):
            year+=1
            str_year = str(year)
            if(len(str_year)<2):
                str_year = "0" + str_year  
                    
        if(str_year=="18"):
            str_year = "8"
                
        tbname = contract_id + month[0] + str_year

        if(date_count < len_count):
            end_date = change_date[date_count]
        else:
#             continue
            end_date = "2017-11-17"

        #資料讀取
        sql = "SELECT * FROM " + tbname + " WHERE Date <= '" + end_date + "' and Date>='" + start_date + "'"

        cursor.execute(sql)
        db.commit()
        table1_data = cursor.fetchall()


        for i in range(len(list(table1_data))):
            contract_name_B2.append(tbname)
            change_B2.append(0)

        contract_B2 = contract_B2.append(pd.DataFrame(list(table1_data)))
        change_B2.pop()
        change_B2.append(1)
            

        #讀取下一個合約的資料
        date_count += 1

        start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

change_B2.pop()
contract_B2['contract_name'] = contract_name_B2
contract_B2['change'] = change_B2

#存檔
columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
contract_B2.to_csv('../日資料/conti/rb_B2.csv',index=None,header=columns)

# 主力合約 (A)

In [9]:
A1_list = ['K','U','F'] #鐵礦石 2013-10-18 /
# A1_list = ['F','K','U'] #玉米2004-10-29
# A1_list = ['U','F','K'] #焦炭 2011-04-15 / 焦煤 2013-03-22 /豆粕2000-7-17 
# A1_list = ['M','Z'] #黃金 2008-01-09
# A1_list = ['Z','M'] #白銀 2012-05-10
# A1_list = ['F','G','H','J','K','M','N','Q','U','V','X','Z'] #銅 2003-01-01 / 鋁2002-01-01
# A1_list = ['G','H','J','K','M','N','Q','U','V','X','Z','F'] #鋅2007-3-26
# A1_list = ['U','V','X','Z','F','G','H','J','K','M','N','Q'] #鉛2011-3-24

In [10]:
A_O2_list = ['U','F','K'] # 鐵礦石A2
A_O3_list = ['F','K','U'] # 鐵礦石A3

# A_O2_list = ['F','K','U'] #焦煤/焦炭A2 
# A_O3_list = ['K','U','F'] #焦煤/焦炭A3

# A_O2_list = ['K','U','F'] #玉米A2 
# A_O3_list = ['U','F','K'] #玉米A3

# A_O2_list = ['Z','M'] #黃金A2
# A_O2_list = ['M','Z'] #白銀A2

#銅 /鋁 1-12
# A_O2_list = ['G','H','J','K','M','N','Q','U','V','X','Z','F'] 
# A_O3_list = ['H','J','K','M','N','Q','U','V','X','Z','F','G']  
# A_O4_list = ['J','K','M','N','Q','U','V','X','Z','F','G','H']  
# A_O5_list = ['K','M','N','Q','U','V','X','Z','F','G','H','J']  
# A_O6_list = ['M','N','Q','U','V','X','Z','F','G','H','J','K'] 
# A_O7_list = ['N','Q','U','V','X','Z','F','G','H','J','K','M']  
# A_O8_list = ['Q','U','V','X','Z','F','G','H','J','K','M','N']  
# A_O9_list = ['U','V','X','Z','F','G','H','J','K','M','N','Q']  
# A_O10_list = ['V','X','Z','F','G','H','J','K','M','N','Q','U']  
# A_O11_list = ['X','Z','F','G','H','J','K','M','N','Q','U','V']  
# A_O12_list = ['Z','F','G','H','J','K','M','N','Q','U','V','X']

#鋅
# A_O2_list = ['H','J','K','M','N','Q','U','V','X','Z','F','G'] 
# A_O3_list = ['J','K','M','N','Q','U','V','X','Z','F','G','H']  
# A_O4_list = ['K','M','N','Q','U','V','X','Z','F','G','H','J']  
# A_O5_list = ['M','N','Q','U','V','X','Z','F','G','H','J','K']  
# A_O6_list = ['N','Q','U','V','X','Z','F','G','H','J','K','M'] 
# A_O7_list = ['Q','U','V','X','Z','F','G','H','J','K','M','N']  
# A_O8_list = ['U','V','X','Z','F','G','H','J','K','M','N','Q']  
# A_O9_list = ['V','X','Z','F','G','H','J','K','M','N','Q','U']  
# A_O10_list = ['X','Z','F','G','H','J','K','M','N','Q','U','V']  
# A_O11_list = ['Z','F','G','H','J','K','M','N','Q','U','V','X']  
# A_O12_list = ['F','G','H','J','K','M','N','Q','U','V','X','Z']

#鉛
# A_O2_list = ['V','X','Z','F','G','H','J','K','M','N','Q','U']
# A_O3_list = ['X','Z','F','G','H','J','K','M','N','Q','U','V']
# A_O4_list = ['Z','F','G','H','J','K','M','N','Q','U','V','X']  
# A_O5_list = ['F','G','H','J','K','M','N','Q','U','V','X','Z'] 
# A_O6_list = ['G','H','J','K','M','N','Q','U','V','X','Z','F']
# A_O7_list = ['H','J','K','M','N','Q','U','V','X','Z','F','G']
# A_O8_list = ['J','K','M','N','Q','U','V','X','Z','F','G','H']
# A_O9_list = ['K','M','N','Q','U','V','X','Z','F','G','H','J']
# A_O10_list = ['M','N','Q','U','V','X','Z','F','G','H','J','K']
# A_O11_list = ['N','Q','U','V','X','Z','F','G','H','J','K','M']
# A_O12_list = ['Q','U','V','X','Z','F','G','H','J','K','M','N']

In [33]:
db = MySQLdb.connect("localhost","root","123456","i_day_data")
db.set_character_set("gbk")

In [13]:
start_date = "2013-10-18"
contract_id = "IOE"

change_date = conti_contract_A1(start_date,contract_id,A1_list,'i_A1.csv',14,19)
conti_contract_A_O(start_date,contract_id,change_date,A_O2_list,'i_A2.csv',14,19)
conti_contract_A_O(start_date,contract_id,change_date,A_O3_list,'i_A3.csv',14,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O4_list,'pb_A4.csv',11,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O5_list,'pb_A5.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O6_list,'pb_A6.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O7_list,'pb_A7.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O8_list,'pb_A8.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O9_list,'pb_A9.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O10_list,'pb_A10.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O11_list,'pb_A11.csv',12,19)
# conti_contract_A_O(start_date,contract_id,change_date,A_O12_list,'pb_A12.csv',12,19)

2014-05-15
IOEK14
2014-09-12
IOEU14
2015-01-13
IOEF15
2015-05-14
IOEK15
2015-09-15
IOEU15
2016-01-14
IOEF16
2016-05-13
IOEK16
2016-09-13
IOEU16
2017-01-13
IOEF17
2017-05-12
IOEK17
2017-09-13
IOEU17
18
2018-01-12
IOEF8
2018-02-01
IOEK8


In [23]:
str(18).split('1')

['', '8']

In [11]:
def conti_contract_A1(start_date,contract_id,A_list,filename,start_year,end_year):

    contract = pd.DataFrame()
    contract_name = []
    change = [1]
    change_date = []

    for year in range(start_year,end_year):
        #修改合約順序
            
        if(year<10):
            str_year = "0" + str(year)
            
        elif(year>=18):
            str_year = str(year).split('1')[1]
            
        else:
            str_year = str(year)
            
        for month in A_list:
            cursor = db.cursor()
            
        #有包含Z的話以下要註解掉    
            if(month=="F"):
                year += 1
                
                if(year < 10):
                    str_year = "0" + str(year)
                else:
                    str_year = str(year)
                    
                if(year>=18):
                    print(str_year)
                    str_year = str(year).split('1')[1]

            tbname = contract_id + month + str_year
            
            if(month=="Z"):
                year += 1
                
                if(year < 10):
                    str_year = "0" + str(year)
                else:
                    str_year = str(year)
                
                if(year>=18):
                    str_year = str(year).split('1')[1]
            
            if(year > 18):
                break;


            #獲取資料
            sql = "SELECT * FROM " + tbname + " WHERE Date >= '" + start_date + "'"
            cursor.execute(sql)
            data = cursor.fetchall()

            if(len(data) == 0):
                continue   

            start_date = (data[len(data) - 1][0] + datetime.timedelta(days=1)).strftime('%Y-%m-%d')
            
            change_date.append(data[len(data) - 1][0].strftime('%Y-%m-%d'))
            
            print(data[len(data) - 1][0])
            print(tbname)

            for i in range(len(data)):
                contract_name.append(tbname)
                change.append(0)

            contract = contract.append(pd.DataFrame(list(data)))
            change.pop()
            change.append(1)
            
            if(start_date >= '2018-02-02'):
                break;

    change.pop()
    contract['contract_name'] = contract_name
    contract['change'] = change

    #存檔
    columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
    contract.to_csv('../日資料/conti/A/' + filename,index=None,header=columns)
    
    return change_date

# 其他合約(A)

In [12]:
def conti_contract_A_O(start_date,contract_id,change_date,A_list,filename,start_year,end_year):

    contract = pd.DataFrame()
    contract_name = []
    change = [1]
    date_count = 0

    for year in range(start_year,end_year):
        
        if(year<10):
            str_year = "0" + str(year)
        elif(year>=18):
            str_year = str(year).split('1')[1]
        else:
            str_year = str(year)
            
        for month in A_list:
            cursor = db.cursor()

        #有包含Z的話以下要註解掉   
            if(month=="F"):
                year+=1

                if(year < 10):
                    str_year = "0" + str(year) 
                else:
                    str_year = str(year)
                    
                if(year>=18):
                    str_year = str(year).split('1')[1]
            
            #之後須砍掉
            if(year > 18):
                break
                
            tbname = contract_id + month + str_year
            
            if(month=="Z"):
                year += 1
                
                if(year < 10):
                    str_year = "0" + str(year)
                else:
                    str_year = str(year)
                
                if(year>=18):
                    str_year = str(year).split('1')[1]

            if(date_count < len(change_date)):
                end_date = change_date[date_count]
                date_count += 1
            else:
                end_date = "2018-02-02"      

            #獲取資料
            sql = "SELECT * FROM " + tbname + " WHERE Date >= '" + start_date + "' and Date <= '" + end_date + "'"
            cursor.execute(sql)
            data = cursor.fetchall()
            start_date = (datetime.datetime.strptime(end_date,'%Y-%m-%d') + datetime.timedelta(days=1)).strftime('%Y-%m-%d')

            
            
            for i in range(len(data)):
                contract_name.append(tbname)
                change.append(0)
            
            contract = contract.append(pd.DataFrame(list(data)))
            change.pop()
            change.append(1)

    change.pop()
    contract['contract_name'] = contract_name
    contract['change'] = change

    #存檔
    columns = ['Date','open','high','low','last','settle','volume','open_int','contract_name','change']
    contract.to_csv('../日資料/conti/A/' + filename ,index=None,header=columns)

# 塞入资料库

In [36]:
db = MySQLdb.connect("localhost","root","123456")
db.set_character_set("gbk")

In [17]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')

In [19]:
DATA_DIR = "../日資料/conti/A/"
for filename in os.listdir(DATA_DIR):
    
    cursor = db.cursor()
    
    contract_id = filename.split("_")[0]
    
    dbname = contract_id + "_day_data_conti"
    
    sql = "CREATE DATABASE IF NOT EXISTS " + dbname

    cursor.execute(sql)
    db.commit()

    cursor = db.cursor()
    
    name = filename.split(".csv")[0]
    
    sql = "CREATE TABLE IF NOT EXISTS " + dbname + "." + name + \
          " (Date DATE primary key,\
             Open FLOAT,\
             High FLOAT,\
             Low FLOAT,\
             Last FLOAT,\
             Settle FLOAT,\
             Volume FLOAT,\
             OI FLOAT,\
             Contract_Name VARCHAR(10),\
             New_Contract INT(2))"

    cursor.execute(sql)
    db.commit()
    
    data = pd.read_csv(DATA_DIR+filename,engine="python")
    
    data = data.where((pd.notnull(data)),None)
    
    values = []

    for i in range(len(data)):
#         if(data.loc[i][0] > str(datetime.datetime(2017,11,17))):
        values.append(tuple(data.loc[i]))
        
    sql = "insert into " + dbname + "." + name + " values (%s,%s,%s,%s,%s,%s,%s,%s,%s,%s)"
    cursor.executemany(sql,values)
    db.commit()
cursor.close()

C:\Users\Philip\Anaconda3\lib\site-packages\ipykernel_launcher.py:12: Warning: (1007, "Can't create database 'i_day_data_conti'; database exists")
  if sys.path[0] == '':


In [43]:
data.loc[1000][0] > str(datetime.datetime(2017,11,17))

True

In [42]:
data.loc[1000][0]

'2017-11-22'

# 刪除空表格

In [7]:
DCE = ['a_1_day_data','a_2_day_data','bb_day_data','c_day_data','cs_day_data','fb_day_data','i_day_data',
     'j_day_data','jd_day_data','jm_day_data','l_day_data','m_day_data','p_day_data','v_day_data','y_day_data'] 

SHFE = ['ag_day_data','al_day_data','au_day_data','bu_day_data','cu_day_data','fu_day_data','hc_day_data'
        ,'ni_day_data','pb_day_data','rb_day_data','ru_day_data','sn_day_data','wr_day_data','zn_day_data']

CCFEX = ['IC_day_data','IF_day_data','IH_day_data','TF_day_data','T_day_data']

CZCE = ['CF_day_data','CY_day_data','FG_day_data','JR_day_data','LR_day_data','MA_day_data','OI_day_data','PM_day_data'
        ,'RI_day_data','RM_day_data','RS_day_data','SF_day_data','SM_day_data','SR_day_data','TA_day_data'
        ,'WH_day_data','ZC_day_data']

In [11]:
for dbname in CZCE:
    print(dbname)
    db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua",dbname)
    db.set_character_set("gbk")
    cursor = db.cursor()
    cursor.execute('SHOW TABLES')
    for table in cursor.fetchall():    
        cursor.execute('SELECT * FROM %s' % table[0])
        if cursor.rowcount == 0:
            cursor.execute('DROP TABLE %s' % table[0])

CF_day_data
CY_day_data
FG_day_data
JR_day_data
LR_day_data
MA_day_data
OI_day_data
PM_day_data
RI_day_data
RM_day_data
RS_day_data
SF_day_data
SM_day_data
SR_day_data
TA_day_data
WH_day_data
ZC_day_data


In [ ]:
db = MySQLdb.connect("47.100.14.137","hyps44413","Nanhua")
db.set_character_set('gbk')